This notebook talks to the lebreVna and the arduino which controls the stepper. 

Example code
https://github.com/jankae/LibreVNA/blob/master/Documentation/UserManual/SCPI_Examples/retrieve_trace_data.py

In [ ]:
#Set up connection, set up sweep, take test sweep, plot 

import time
from libreVNA import libreVNA
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget


#VNA parameters
startFreq = 50 #MHz
stopFreq = 1050 #MHz


# Create the control instance
vna = libreVNA('localhost', 19542)

# Quick connection check (should print "LibreVNA-GUI")
print(vna.query("*IDN?"))

# Make sure we are connecting to a device (just to be sure, with default settings the LibreVNA-GUI auto-connects)
vna.cmd(":DEV:CONN")

dev = vna.query(":DEV:CONN?")
if dev == "Not connected":
    print("Not connected to any device, aborting")
    exit(-1)
else:
    print("Connected to "+dev)
# Simple trace data extraction

print("Setting up the sweep...")
vna.cmd(":DEV:MODE VNA")
vna.cmd(":VNA:SWEEP FREQUENCY")
vna.cmd(":VNA:STIM:LVL 0")
vna.cmd(":VNA:ACQ:IFBW 2500") #IF bandwidth. Basically RBW
vna.cmd(":VNA:ACQ:AVG 1")
vna.cmd(":VNA:ACQ:POINTS 4001") #data points. max = 4501
vna.cmd(":VNA:FREQuency:START "+ str(int(startFreq*1e6)))
vna.cmd(":VNA:FREQuency:STOP " + str(int(stopFreq*1e6)))

# wait for the sweep to finish
print("Waiting for the sweep to finish...")
while vna.query(":VNA:ACQ:FIN?") == "FALSE":
    time.sleep(0.1)

S11 = vna.parse_trace_data(vna.query(":VNA:TRACE:DATA? S11"))
freqs = [data[0]/1e6 for data in S11] #MHz
s11Comp = [data[1] for data in S11]
S22 = vna.parse_trace_data(vna.query(":VNA:TRACE:DATA? S22"))
s22Comp = [data[1] for data in S22]


'''print("Setting up the sweep...")
vna.cmd(":DEV:MODE VNA")
vna.cmd(":VNA:SWEEP FREQUENCY")
vna.cmd(":VNA:STIM:LVL 0")
vna.cmd(":VNA:ACQ:IFBW 5000") #IF bandwidth. Basically RBW
vna.cmd(":VNA:ACQ:AVG 1")
vna.cmd(":VNA:ACQ:POINTS 4001") #data points. max = 4501
vna.cmd(":VNA:FREQuency:START "+ str(int(900*1e6)))
vna.cmd(":VNA:FREQuency:STOP " + str(int(920*1e6)))

# wait for the sweep to finish
print("Waiting for the sweep to finish...")
while vna.query(":VNA:ACQ:FIN?") == "FALSE":
    time.sleep(0.1)

S11 = vna.parse_trace_data(vna.query(":VNA:TRACE:DATA? S11"))
freqsHR = [data[0]/1e6 for data in S11] #MHz
s11Comp = [data[1] for data in S11]
S22 = vna.parse_trace_data(vna.query(":VNA:TRACE:DATA? S22"))
s22CompHR = [data[1] for data in S22]
'''

if 0:
    plt.figure()
    plt.plot(freqs, np.real(s22Comp), label = 'real')
    plt.plot(freqs, np.imag(s22Comp), label = 'img')
    plt.plot(freqs, np.absolute(s22Comp), label = 'mag')
    plt.xlabel('Frequency [MHz]')
    plt.ylabel('Linear S-parameters [None]')
    plt.legend()

if 1:
    plt.figure()
    plt.plot(freqsHR, 20*np.log10(np.absolute(s22CompHR)), label = 'magnitude')
    #plt.scatter(freqsHR, 20*np.log10(np.absolute(s22CompHR)), label = 'magnitude')
    plt.plot(freqs, 20*np.log10(np.absolute(s22Comp)), label = 'magnitude')
    plt.scatter(freqs, 20*np.log10(np.absolute(s22Comp)), label = 'magnitude')
    plt.xlabel('Frequency [MHz]')
    plt.ylabel('S-parameters [dB]')
    #plt.legend()



In [ ]:
if 1:
    plt.figure()
    plt.grid()
    plt.plot(freqsHR, 20*np.log10(np.absolute(s22CompHR)), 'b', label = 'High res')
    #plt.scatter(freqsHR, 20*np.log10(np.absolute(s22CompHR)), color = 'b', label = 'magnitude')
    #plt.plot(freqs, 20*np.log10(np.absolute(s22Comp)), 'r', label = 'magnitude')
    plt.scatter(freqs, 20*np.log10(np.absolute(s22Comp)), color = 'b', label = 'low res')
    
    plt.plot(freqsHR, 20*np.log10(np.absolute(s22CompHRb)), 'r', label = 'High res, p2')
    #plt.scatter(freqsHR, 20*np.log10(np.absolute(s22CompHR)), color = 'b', label = 'magnitude')
    #plt.plot(freqs, 20*np.log10(np.absolute(s22Comp)), 'r', label = 'magnitude')
    plt.scatter(freqs, 20*np.log10(np.absolute(s22Compb)), color = 'r', label = 'low res, p2')
    plt.xlabel('Frequency [MHz]')
    plt.ylabel('S11 (Big vivaldi) [dB]')
    plt.legend()
    #plt.xlim(910, 912)
    #plt.ylim(-35, -20)


In [ ]:
vna.cmd(":VNA:SWEEP FREQUENCY")
vna.cmd(":VNA:ACQ:POINTS 4000") #data points. max = 4501
vna.cmd(":VNA:FREQuency:START 250000000")
vna.cmd(":VNA:FREQuency:STOP 1050000000")
# wait for the sweep to finish
print("Waiting for the sweep to finish...")
time.sleep(4)
while vna.query(":VNA:ACQ:FIN?") == "FALSE":
    time.sleep(0.1)

S11 = vna.parse_trace_data(vna.query(":VNA:TRACE:DATA? S11"))
S12 = vna.parse_trace_data(vna.query(":VNA:TRACE:DATA? S12")) #S12 = S21 by recaprocity 
S22 = vna.parse_trace_data(vna.query(":VNA:TRACE:DATA? S22"))

print(S11[3])
print(S12[3])
print(S22[3])